<a href="https://colab.research.google.com/github/dibyasha2021/mini_project-/blob/main/Combine_Dataset_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd 
import seaborn as sns

import librosa
import librosa.display
import IPython.display as ipd
from librosa.display import specshow
from IPython.display import Audio

from glob import glob
import os

import warnings
warnings.filterwarnings('ignore')


from scipy import mean


## Audio Extraction

In [ ]:
# Audio Destination

_fear=r"E:\Project & Research\mini project\dataset1\TESS Toronto emotional speech set data\OAF\OAF_fear\*.wav"
_happy=r"E:\Project & Research\mini project\dataset1\TESS Toronto emotional speech set data\OAF\OAF_happy\*.wav"
_neutral=r"E:\Project & Research\mini project\dataset1\TESS Toronto emotional speech set data\OAF\OAF_neutral\*.wav"
_sad=r"E:\Project & Research\mini project\dataset1\TESS Toronto emotional speech set data\OAF\OAF_Sad\*.wav"
_angry=r"E:\Project & Research\mini project\dataset1\TESS Toronto emotional speech set data\OAF\OAF_angry\*.wav"
_disgust=r"E:\Project & Research\mini project\dataset1\TESS Toronto emotional speech set data\OAF\OAF_disgust\*.wav"
_PS=r"E:\Project & Research\mini project\dataset1\TESS Toronto emotional speech set data\OAF\OAF_Pleasant_surprise\*.wav"

In [ ]:
# Accessing the data 
file_path_fear=list(glob(_fear))[0:200] #1 
file_path_happy=list(glob(_happy))[0:200] #2
file_path_neutral =list(glob(_neutral))[0:200] #3
file_path_sad =list(glob(_sad))[0:200] #4
file_path_angry =list(glob(_angry))[0:200] #5
file_path_disgust =list(glob(_disgust))[0:200] #6
file_path_PS =list(glob(_PS))[0:200] #7

In [ ]:
# function to extract the waveplot(data , sample rate) from a given path 

def waveplot(path):
    
    #plt.figure(figsize=(10,4))
    
    data, sample_rate = librosa.load(path)
    
    #librosa.display.waveshow(data, sr=sample_rate)
    
    #plt.show()
    
    return data,sample_rate


## MFCCs Extraction 

In [ ]:
#function to calculate mean 13 MFCCs of each audio 
def MFCCS_feature(Audio,sr):
    #extracting the MFCC
    
    mfccs=librosa.feature.mfcc(Audio,n_mfcc=13,sr=sr)
    
    #print(mfccs)
    #from spectrogram for MFCC
    
    #Visualize MFCCs
    #plt.figure(figsize =(25,10))
    #librosa.display.specshow(mfccs,x_axis="time",sr=sample_rate)
    #plt.colorbar(format="%+2.f")
    #plt.show()
    
    # calculating mean of each coefficient from all the frames
    
    coef_mean=[]
    for i in range(13):
        coef_mean.append(mean(mfccs[i]))
    
    #coef_mean
    
    return coef_mean
    

## Spectral Centroid

In [ ]:
#function to calculate the Spectral Centroid(Mean,max,min,IQR,std) 
def Spectral_freq(Audio,FRAME_SIZE,hop_LEN):
    
    #time series input
    cent=librosa.feature.spectral_centroid(Audio, sr=sr)[0]
    
    #from spectrogram input (using variable bin frequencies)
    freqs, times, D = librosa.reassigned_spectrogram(Audio,n_fft=FRAME_SIZE,reassign_frequencies=False, fill_nan=True)
    
    cent2=librosa.feature.spectral_centroid(S=np.abs(D), freq=freqs)[0]
    
    #plt.plot(cent2)
    #plt.xlabel('Frame number')
    #plt.ylabel('frequency (Hz)')
    #plt.title('Spectral centroids')
    #plt.show()
    #print(cent2)
    #print(len(cent2))
    
    S_mean= cent2.mean()  # Mean of SC of frames
    
    S_max=cent2.max() # max SC of Audio
    
    S_min=cent2.min() # min SC of audio
    
    q3, q1 = np.percentile(cent2, [75 ,25])
    iqr = q3 - q1
    
    S_iqr=iqr  # IQR SC of Audio
    
    S_sd = np.std(cent2)  # standard Deviation of SC in audio
    
    return S_mean,S_max,S_min,S_iqr,S_sd
    

## Root Mean Square Energry 

In [ ]:
#function to normalize the given audio

def total_rms(Audiofile):
    
    R=0
    for j in Audiofile:
        R+=(j)**2
        rms=(R/len(Audiofile))**(0.5)


    return rms
# function to calculate the Root Mean Square Energry of an audio

def RMS_(Audio_data_N,Frame_LEN,hop_LEN):
    
    #RMS_of each frame :
    Rms_Audio= librosa.feature.rms(Audio_data_N,frame_length =Frame_LEN,hop_length = hop_LEN)
    
    RMS_mean= Rms_Audio.mean()  # Mean of rms of frames 
       
    RMS_max=Rms_Audio.max()  #max of audio
   
    RMS_min=Rms_Audio.min()   #min of audio
    
    q3, q1 = np.percentile(Rms_Audio, [75 ,25])
    iqr = q3 - q1
        
    RMS_iqr=iqr  # IQR of RMS 
        
    RMS_sd = np.std(Rms_Audio)  # standard Deviation of RMS value
    
    
    return RMS_mean,RMS_max,RMS_min,RMS_iqr,RMS_sd 

In [ ]:
#MFCCs Audio in old-fear category ---
#01

# list to store the statistical mearsure of Spectral Centroid 

SC_MEAN=[]
SC_MAX=[]
SC_MIN=[]
SC_IQR=[]
SC_STD=[]

# list to store the statistical mearsure of RMSE 

RMS_MEAN=[]
RMS_MAX=[]
RMS_MIN=[]
RMS_IQR=[]
RMS_STD=[]

# List of all the coefficient 

maincoeff_fear=[]

for i in range(200):
    # MFCCs
    audio_data,sr=waveplot(file_path_fear[i])
    MFCCs_coeff=MFCCS_feature(audio_data,sr)
    maincoeff_fear.append(MFCCs_coeff)
    
    #RMSE
    R=total_rms(audio_data)
    Audio_data_N =audio_data/R #for normalized each audio in old-fear category
    
    MEAN,MAX,MIN,IQR,SD=RMS_(Audio_data_N,1024,512)
    
    RMS_MEAN.append(MEAN)
    RMS_MAX.append(MAX)
    RMS_MIN.append(MIN)
    RMS_IQR.append(IQR)
    RMS_STD.append(SD)
    
    # Spectral Centroid
    S_MEAN,S_MAX,S_MIN,S_iqr,S_sd=Spectral_freq(audio_data,1024,512 )
    SC_MEAN.append(S_MEAN)
    SC_MAX.append(S_MAX)
    SC_MIN.append(S_MIN)
    SC_IQR.append(S_iqr)
    SC_STD.append(S_sd)
    
# Making the Dictionary of RMS Statistical values and Spectral Centroid statistical Values 

dic={"MEAN_RMSE":RMS_MEAN,"MAX_RMSE":RMS_MAX,"MIN_RMSE":RMS_MIN,"IQR_RMSE":RMS_IQR,"STD_RMSE":RMS_STD,"MEAN_SC":SC_MEAN,"MAX_SC":SC_MAX,"MIN_SC":SC_MIN,"IQR_SC":SC_IQR,"STD_SC":SC_STD}

Df_r_s=pd.DataFrame(data=dic)

#print(MFCCs_coeff)    
#maincoeff_fear

# Making a Dataframe of the given Audio 

Df_mfcc=pd.DataFrame(data=maincoeff_fear,columns=("C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13"))
Df_mfcc['Label']="FEAR"
Df_fear=pd.concat([Df_r_s, Df_mfcc], axis=1)

Df_fear


,MEAN_RMSE,MAX_RMSE,MIN_RMSE,IQR_RMSE,STD_RMSE,MEAN_SC,MAX_SC,MIN_SC,IQR_SC,STD_SC,...,C5,C6,C7,C8,C9,C10,C11,C12,C13,Label
0,0.771957,2.092660,0.014741,1.228814,0.629168,1962.031387,6378.780076,402.558634,608.917698,1509.051985,...,2.283087,3.844167,-14.228564,0.269647,-11.322652,4.499730,-13.458061,-6.326655,-4.551631,FEAR
1,0.817040,2.150111,0.027914,0.838567,0.573831,1942.036062,6370.429580,510.573035,557.204637,1538.834203,...,-7.444870,5.448284,-12.487878,0.189825,-16.319094,1.524716,-21.328081,-3.633686,-6.368850,FEAR
2,0.785567,2.242551,0.017252,1.188739,0.613742,2442.161223,6223.610055,450.629524,1456.653181,1800.796830,...,7.653680,0.787712,-13.500839,-1.832727,-12.877916,1.519425,-18.313948,-6.033486,-3.560616,FEAR
3,0.781189,2.286831,0.012800,1.080863,0.618558,2357.061960,6657.249797,385.559075,1540.755298,1791.091198,...,-2.028609,3.328899,-12.075669,-0.014596,-15.432942,4.545129,-14.551147,-4.555892,-8.357164,FEAR
4,0.817185,1.942002,0.026914,0.911707,0.565479,1914.672326,5802.151158,460.611570,743.397902,1485.758096,...,8.218389,1.294082,-7.010327,2.554072,-14.801518,-2.347218,-18.474848,-5.801582,-2.754475,FEAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.801847,2.412958,0.026136,1.037603,0.594563,2355.636956,6081.606228,504.023823,1488.443378,1639.493170,...,0.047164,1.471613,-11.291485,2.138622,-17.013527,3.412043,-17.250145,-3.371418,-4.460457,FEAR
196,0.868523,1.786907,0.018345,0.797532,0.480194,2007.627002,6228.612730,666.566043,743.202017,1348.320595,...,0.376796,3.849858,-17.650448,-12.475883,-16.672514,-3.814930,-24.633806,-11.657144,-6.282569,FEAR
197,0.809790,2.260921,0.039270,0.874796,0.579446,2575.496240,6286.030384,730.930516,1475.838919,1742.303407,...,5.164965,-1.687155,-14.073120,-2.114274,-17.601795,4.441251,-17.058287,-5.866043,-2.122936,FEAR
198,0.864398,1.968902,0.040735,0.773597,0.488905,2010.014294,6296.354739,745.603495,690.193091,1460.396724,...,-2.542353,-0.606435,-16.322857,-1.802548,-21.373558,-1.870263,-21.539740,-7.256168,-9.799089,FEAR


In [ ]:
#MFCCs Audio in old-happy category ---
#02
# list to store the statistical mearsure of Spectral Centroid 

SC_MEAN=[]
SC_MAX=[]
SC_MIN=[]
SC_IQR=[]
SC_STD=[]

# list to store the statistical mearsure of RMSE 

RMS_MEAN=[]
RMS_MAX=[]
RMS_MIN=[]
RMS_IQR=[]
RMS_STD=[]

# List of all the coefficient 
maincoeff_happy=[]

for i in range(200):

    audio_data,sr=waveplot(file_path_happy[i])
    MFCCs_coeff=MFCCS_feature(audio_data,sr)
    maincoeff_happy.append(MFCCs_coeff)
    
    #RMSE
    R=total_rms(audio_data)
    Audio_data_N =audio_data/R #for normalized each audio in old-fear category
    
    MEAN,MAX,MIN,IQR,SD=RMS_(Audio_data_N,1024,512)
    
    RMS_MEAN.append(MEAN)
    RMS_MAX.append(MAX)
    RMS_MIN.append(MIN)
    RMS_IQR.append(IQR)
    RMS_STD.append(SD)
    
    # Spectral Centroid
    S_MEAN,S_MAX,S_MIN,S_iqr,S_sd=Spectral_freq(audio_data,1024,512 )
    SC_MEAN.append(S_MEAN)
    SC_MAX.append(S_MAX)
    SC_MIN.append(S_MIN)
    SC_IQR.append(S_iqr)
    SC_STD.append(S_sd)
    
# Making the Dictionary of RMS Statistical values and Spectral Centroid statistical Values 

dic={"MEAN_RMSE":RMS_MEAN,"MAX_RMSE":RMS_MAX,"MIN_RMSE":RMS_MIN,"IQR_RMSE":RMS_IQR,"STD_RMSE":RMS_STD,"MEAN_SC":SC_MEAN,"MAX_SC":SC_MAX,"MIN_SC":SC_MIN,"IQR_SC":SC_IQR,"STD_SC":SC_STD}

Df_r_s=pd.DataFrame(data=dic)

    
    
#print(MFCCs_coeff)    
#maincoeff_happy

# Making a Dataframe of the given Audio 

Df_mfcc=pd.DataFrame(data=maincoeff_happy,columns=("C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13"))
Df_mfcc['Label']="HAPPY"
#Df_mfcc

Df_happy=pd.concat([Df_r_s, Df_mfcc], axis=1)

Df_happy

,MEAN_RMSE,MAX_RMSE,MIN_RMSE,IQR_RMSE,STD_RMSE,MEAN_SC,MAX_SC,MIN_SC,IQR_SC,STD_SC,...,C5,C6,C7,C8,C9,C10,C11,C12,C13,Label
0,0.767502,2.473227,0.038616,1.062319,0.633592,2046.277883,6416.553825,357.928887,1155.463922,1590.275667,...,3.263430,8.461172,-14.175427,3.392377,-9.518599,-0.258332,-9.737500,-6.645960,-1.898563,HAPPY
1,0.777149,3.290190,0.020879,0.821841,0.621490,1845.282555,6788.707600,405.401845,904.173254,1653.556409,...,-1.431095,6.231369,-8.793474,-1.100019,-14.364656,-0.092157,-13.406323,-3.281358,-3.081315,HAPPY
2,0.753110,2.824213,0.021691,0.956084,0.654087,2464.589573,6623.296621,399.444367,2371.268846,1999.660398,...,2.680684,1.273895,-7.646893,-1.344968,-9.695471,4.321653,-13.581964,-2.412816,-2.886339,HAPPY
3,0.799897,2.367037,0.033516,0.963621,0.596700,2118.438739,6579.100888,385.415820,1097.843593,1734.094454,...,-0.290889,6.861618,-14.605416,3.281878,-10.715553,-0.994758,-9.576382,-6.511749,-5.932707,HAPPY
4,0.779869,2.205204,0.005586,1.109045,0.623078,1961.253965,6504.067992,387.505068,925.776352,1603.812756,...,3.820580,0.353219,-6.924337,-3.891668,-21.467379,-5.510629,-15.031913,-4.559387,-4.296011,HAPPY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.768333,2.064403,0.037476,0.918362,0.633361,2252.879588,6612.780781,418.077069,1550.625317,1689.658627,...,3.845813,-1.218042,-12.434600,0.530458,-13.838544,0.080987,-13.348592,-4.825954,0.349469,HAPPY
196,0.837743,2.548402,0.007835,0.722477,0.537338,1924.819000,6473.723863,475.733827,977.229242,1628.540809,...,3.043687,5.571729,-13.639851,-7.592187,-10.399581,-8.919054,-15.039314,-11.906637,-4.227028,HAPPY
197,0.778482,2.503490,0.046309,0.859985,0.619984,2591.837953,6818.347095,448.090001,2294.351736,1924.442614,...,0.514156,-1.003985,-14.362069,-0.924537,-14.030045,0.823688,-10.376677,-6.423187,-0.904334,HAPPY
198,0.814804,2.511150,0.034307,0.749063,0.571009,2043.076149,6737.128810,456.659870,1016.530673,1666.249625,...,-2.135378,0.858193,-16.032568,1.592969,-20.101488,-2.075467,-13.757592,-9.034714,-5.446064,HAPPY


In [ ]:
#MFCCs Audio in old-neutral category ---
#03
# list to store the statistical mearsure of Spectral Centroid 

SC_MEAN=[]
SC_MAX=[]
SC_MIN=[]
SC_IQR=[]
SC_STD=[]

# list to store the statistical mearsure of RMSE 

RMS_MEAN=[]
RMS_MAX=[]
RMS_MIN=[]
RMS_IQR=[]
RMS_STD=[]

# List of all the coefficient 
maincoeff_neutral=[]

for i in range(200):

    audio_data,sr=waveplot(file_path_neutral[i])
    MFCCs_coeff=MFCCS_feature(audio_data,sr)
    maincoeff_neutral.append(MFCCs_coeff)
    
     
    #RMSE
    R=total_rms(audio_data)
    Audio_data_N =audio_data/R #for normalized each audio in old-fear category
    
    MEAN,MAX,MIN,IQR,SD=RMS_(Audio_data_N,1024,512)
    
    RMS_MEAN.append(MEAN)
    RMS_MAX.append(MAX)
    RMS_MIN.append(MIN)
    RMS_IQR.append(IQR)
    RMS_STD.append(SD)
    
    # Spectral Centroid
    S_MEAN,S_MAX,S_MIN,S_iqr,S_sd=Spectral_freq(audio_data,1024,512 )
    SC_MEAN.append(S_MEAN)
    SC_MAX.append(S_MAX)
    SC_MIN.append(S_MIN)
    SC_IQR.append(S_iqr)
    SC_STD.append(S_sd)
    
# Making the Dictionary of RMS Statistical values and Spectral Centroid statistical Values 

dic={"MEAN_RMSE":RMS_MEAN,"MAX_RMSE":RMS_MAX,"MIN_RMSE":RMS_MIN,"IQR_RMSE":RMS_IQR,"STD_RMSE":RMS_STD,"MEAN_SC":SC_MEAN,"MAX_SC":SC_MAX,"MIN_SC":SC_MIN,"IQR_SC":SC_IQR,"STD_SC":SC_STD}

Df_r_s=pd.DataFrame(data=dic)
    
 
#print(MFCCs_coeff)    
#maincoeff_neutral

# Making a Dataframe of the given Audio 

Df_mfcc=pd.DataFrame(data=maincoeff_neutral,columns=("C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13"))
Df_mfcc['Label']="NEUTRAl"
#Df_mfcc

Df_neutral=pd.concat([Df_r_s, Df_mfcc], axis=1)

Df_neutral

,MEAN_RMSE,MAX_RMSE,MIN_RMSE,IQR_RMSE,STD_RMSE,MEAN_SC,MAX_SC,MIN_SC,IQR_SC,STD_SC,...,C5,C6,C7,C8,C9,C10,C11,C12,C13,Label
0,0.844518,1.931861,0.055054,1.005513,0.524973,1595.766961,6149.763292,433.019290,643.770298,1274.390536,...,13.853488,11.472582,-7.577172,-4.564630,-11.390241,5.582869,-7.845232,2.649279,-2.582675,NEUTRAl
1,0.857299,2.070387,0.082226,0.878260,0.506017,1724.549927,6171.117149,519.317469,532.413447,1312.759917,...,2.347546,6.509349,-1.266473,-6.623940,-13.517923,0.926941,-18.526688,7.682671,-2.185151,NEUTRAl
2,0.837410,1.817632,0.064815,1.084004,0.542475,2214.400264,6480.318535,442.191031,714.171415,1745.822152,...,11.320130,3.866915,-6.612742,-7.324147,-12.202477,-2.339531,-12.829696,4.567454,-0.602423,NEUTRAl
3,0.824791,2.586818,0.045686,0.911271,0.558149,1997.150293,5885.237950,393.492234,893.490694,1600.744030,...,8.645407,8.761402,-7.624350,-3.235050,-12.102530,2.750155,-7.041051,2.460948,-4.133174,NEUTRAl
4,0.865469,1.702513,0.065740,0.895829,0.494549,1536.758230,6313.675821,344.985322,583.311741,1447.963654,...,13.646284,5.394039,-0.239225,0.157162,-15.032776,-4.660875,-14.879074,-2.953203,-7.944446,NEUTRAl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.884972,1.841336,0.075229,0.657082,0.458763,1555.665409,6127.345484,494.994456,539.259374,1405.184988,...,16.391506,17.359634,-7.136119,-12.837242,-14.185144,-0.024492,-12.757754,1.202288,-8.047926,NEUTRAl
196,0.844254,2.113174,0.040385,0.927434,0.530540,2208.310715,6399.882694,475.341140,914.929565,1847.242568,...,13.275624,7.887555,-12.801099,-7.758018,-13.846498,2.180310,-11.244412,2.635962,-6.367472,NEUTRAl
197,0.881232,1.874918,0.073064,0.814470,0.468844,1695.015350,6123.874605,403.463309,654.009090,1464.039455,...,10.676008,4.624894,-4.987256,-7.612170,-16.263466,-5.008581,-8.409937,2.903893,-7.417601,NEUTRAl
198,0.833213,2.132004,0.094921,0.813072,0.544145,1793.693206,5930.482171,487.237085,865.440711,1609.352660,...,17.801802,9.134454,1.705987,-10.167632,-10.298632,-0.804567,-9.597704,3.074432,-4.286202,NEUTRAl


In [ ]:
#MFCCs Audio in old-sad category ---
#04

# list to store the statistical mearsure of Spectral Centroid 

SC_MEAN=[]
SC_MAX=[]
SC_MIN=[]
SC_IQR=[]
SC_STD=[]

# list to store the statistical mearsure of RMSE 

RMS_MEAN=[]
RMS_MAX=[]
RMS_MIN=[]
RMS_IQR=[]
RMS_STD=[]

# List of all the coefficient 
maincoeff_sad=[]

for i in range(200):

    audio_data,sr=waveplot(file_path_sad[i])
    MFCCs_coeff=MFCCS_feature(audio_data,sr)
    maincoeff_sad.append(MFCCs_coeff)
    
    #RMSE
    R=total_rms(audio_data)
    Audio_data_N =audio_data/R #for normalized each audio in old-fear category
    
    MEAN,MAX,MIN,IQR,SD=RMS_(Audio_data_N,1024,512)
    
    RMS_MEAN.append(MEAN)
    RMS_MAX.append(MAX)
    RMS_MIN.append(MIN)
    RMS_IQR.append(IQR)
    RMS_STD.append(SD)
    
    # Spectral Centroid
    S_MEAN,S_MAX,S_MIN,S_iqr,S_sd=Spectral_freq(audio_data,1024,512 )
    SC_MEAN.append(S_MEAN)
    SC_MAX.append(S_MAX)
    SC_MIN.append(S_MIN)
    SC_IQR.append(S_iqr)
    SC_STD.append(S_sd)
    
# Making the Dictionary of RMS Statistical values and Spectral Centroid statistical Values 

dic={"MEAN_RMSE":RMS_MEAN,"MAX_RMSE":RMS_MAX,"MIN_RMSE":RMS_MIN,"IQR_RMSE":RMS_IQR,"STD_RMSE":RMS_STD,"MEAN_SC":SC_MEAN,"MAX_SC":SC_MAX,"MIN_SC":SC_MIN,"IQR_SC":SC_IQR,"STD_SC":SC_STD}

Df_r_s=pd.DataFrame(data=dic)
    
    
       
#print(MFCCs_coeff)    
#maincoeff_sad

# Making a Dataframe of the given Audio 

Df_mfcc=pd.DataFrame(data=maincoeff_sad,columns=("C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13"))
Df_mfcc['Label']="SAD"
#Df_mfcc

Df_sad=pd.concat([Df_r_s, Df_mfcc], axis=1)

Df_sad

,MEAN_RMSE,MAX_RMSE,MIN_RMSE,IQR_RMSE,STD_RMSE,MEAN_SC,MAX_SC,MIN_SC,IQR_SC,STD_SC,...,C5,C6,C7,C8,C9,C10,C11,C12,C13,Label
0,0.812385,2.522521,0.073806,0.963967,0.578100,1754.158960,6017.843356,312.948904,874.059684,1452.498007,...,15.759771,16.283089,-13.166125,-9.107759,-11.867360,4.691811,-10.124372,8.115974,-6.235575,SAD
1,0.766224,2.833498,0.025897,0.608707,0.640712,1675.041887,5959.386731,527.502426,678.738484,1544.145337,...,9.031152,12.269159,-8.349506,-9.783082,-13.096498,5.476128,-11.960439,6.352079,-5.031146,SAD
2,0.789031,2.294582,0.019638,1.034099,0.611882,1875.130729,6065.658376,457.020397,1281.726724,1650.230288,...,19.353167,10.682536,-16.187931,-18.289804,-13.460554,5.962748,-14.620941,5.269866,-4.817742,SAD
3,0.813062,2.343432,0.072785,0.844221,0.576718,1692.100655,5974.972581,459.515799,900.370669,1499.504745,...,16.794546,14.558558,-8.601521,-6.337084,-13.422338,4.586628,-6.186538,5.645577,-8.326189,SAD
4,0.799866,1.974489,0.013133,1.133354,0.596652,1516.984425,5813.603932,386.096498,877.406061,1408.718530,...,17.063984,10.468684,-0.115961,0.818456,-10.973750,-1.332384,-11.412591,-1.435256,-10.690149,SAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.828865,2.510878,0.078524,0.836053,0.557287,1771.947235,6095.410227,425.622148,809.521985,1596.647105,...,15.116753,10.338654,-11.097422,-12.286369,-12.496275,7.412939,-15.104959,5.683190,-4.719932,SAD
196,0.802405,2.413729,0.037015,0.950631,0.595053,1712.238317,6483.905204,544.068830,963.193165,1598.957299,...,22.752008,16.375408,-13.722204,-15.826996,-13.256080,11.509844,-14.488193,6.815586,-3.367639,SAD
197,0.800074,2.128931,0.029113,1.125879,0.596281,1959.538853,5967.186244,465.921234,1797.210015,1715.613759,...,16.651215,8.695662,-5.359285,-13.437304,-8.763243,6.065662,-12.051786,4.778674,-4.896595,SAD
198,0.800052,3.269948,0.063868,0.636351,0.593350,1702.619260,6038.736991,465.957507,771.765682,1632.923278,...,16.133327,12.658668,-10.082213,-15.001758,-16.346869,4.018665,-13.628448,1.707705,-6.912366,SAD


In [ ]:
#MFCCs & its Derivatives of first Audio in old-angry category ---
#05
# list to store the statistical mearsure of Spectral Centroid 

SC_MEAN=[]
SC_MAX=[]
SC_MIN=[]
SC_IQR=[]
SC_STD=[]

# list to store the statistical mearsure of RMSE 

RMS_MEAN=[]
RMS_MAX=[]
RMS_MIN=[]
RMS_IQR=[]
RMS_STD=[]

# List of all the coefficient 
maincoeff_angry=[]

for i in range(200):

    audio_data,sr=waveplot(file_path_angry[i])
    MFCCs_coeff=MFCCS_feature(audio_data,sr)
    maincoeff_angry.append(MFCCs_coeff)
    
    #RMSE
    R=total_rms(audio_data)
    Audio_data_N =audio_data/R #for normalized each audio in old-fear category
    
    MEAN,MAX,MIN,IQR,SD=RMS_(Audio_data_N,1024,512)
    
    RMS_MEAN.append(MEAN)
    RMS_MAX.append(MAX)
    RMS_MIN.append(MIN)
    RMS_IQR.append(IQR)
    RMS_STD.append(SD)
    
    # Spectral Centroid
    S_MEAN,S_MAX,S_MIN,S_iqr,S_sd=Spectral_freq(audio_data,1024,512 )
    SC_MEAN.append(S_MEAN)
    SC_MAX.append(S_MAX)
    SC_MIN.append(S_MIN)
    SC_IQR.append(S_iqr)
    SC_STD.append(S_sd)
    
# Making the Dictionary of RMS Statistical values and Spectral Centroid statistical Values 

dic={"MEAN_RMSE":RMS_MEAN,"MAX_RMSE":RMS_MAX,"MIN_RMSE":RMS_MIN,"IQR_RMSE":RMS_IQR,"STD_RMSE":RMS_STD,"MEAN_SC":SC_MEAN,"MAX_SC":SC_MAX,"MIN_SC":SC_MIN,"IQR_SC":SC_IQR,"STD_SC":SC_STD}

Df_r_s=pd.DataFrame(data=dic)
       
#print(MFCCs_coeff)    
#maincoeff_angry

# Making a Dataframe of the given Audio 

Df_mfcc=pd.DataFrame(data=maincoeff_angry,columns=("C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13"))
Df_mfcc['Label']="ANGRY"
Df_mfcc

Df_angry=pd.concat([Df_r_s, Df_mfcc], axis=1)

Df_angry

,MEAN_RMSE,MAX_RMSE,MIN_RMSE,IQR_RMSE,STD_RMSE,MEAN_SC,MAX_SC,MIN_SC,IQR_SC,STD_SC,...,C5,C6,C7,C8,C9,C10,C11,C12,C13,Label
0,0.706625,2.693167,0.010510,1.039915,0.699967,1915.773645,5759.607978,381.017900,1024.696770,1210.213729,...,-5.176102,7.989569,-25.547773,-6.490075,-10.548781,-2.805863,-0.165465,1.098092,1.624355,ANGRY
1,0.759638,2.503734,0.033802,1.080511,0.646183,1916.538265,5679.423049,348.713775,1097.608031,1303.097085,...,-10.965109,10.690770,-10.050633,-0.911897,-12.894197,3.262048,-9.962346,10.416118,-1.206751,ANGRY
2,0.688755,2.999988,0.023591,1.001644,0.718896,2647.247580,6035.300962,346.360535,1912.599660,1622.141811,...,-1.557886,1.512646,-19.856451,-1.872496,-13.751393,-0.483411,-13.203755,8.220826,-1.319538,ANGRY
3,0.706647,2.693190,0.005039,0.896297,0.699476,2224.534968,6543.265685,387.454776,1210.268187,1606.869701,...,-4.485830,7.391958,-22.273468,-7.113718,-13.866476,-2.610887,-2.195202,1.928133,0.075706,ANGRY
4,0.790981,2.712722,0.034718,0.804505,0.599446,2060.166316,6189.486476,341.616255,1511.698143,1366.502354,...,3.426946,1.198985,-24.629528,7.922730,-19.956839,-11.991178,-11.831434,-1.378742,-5.277905,ANGRY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.722788,2.764976,0.011718,1.040594,0.681758,2289.321346,6133.315781,536.900228,1565.204749,1315.145187,...,-6.625676,-1.070136,-18.455379,0.557779,-13.652376,1.555675,-12.595828,10.249820,2.219247,ANGRY
196,0.763807,2.185274,0.008273,1.124724,0.635545,1780.906441,5706.574590,629.589203,651.651967,1155.459896,...,2.734374,18.490767,-30.310190,-22.025560,-20.667316,-3.952531,-5.495463,-2.851287,3.306413,ANGRY
197,0.743695,2.507301,0.012833,0.870395,0.664298,2516.505282,6155.982611,495.553269,1953.323279,1602.230416,...,-6.654065,-4.046296,-26.178345,2.667480,-12.160625,0.597317,-7.608262,1.150036,-8.605457,ANGRY
198,0.712564,2.921245,0.028391,0.705973,0.693037,1956.566299,5775.333451,491.902033,987.887018,1232.154768,...,-11.381028,4.277552,-18.898006,-3.349270,-12.696109,-5.254924,-8.460628,7.151527,-7.080307,ANGRY


In [ ]:
#MFCCs & its Derivatives of first Audio in old-disgust category ---
#06
# list to store the statistical mearsure of Spectral Centroid 

SC_MEAN=[]
SC_MAX=[]
SC_MIN=[]
SC_IQR=[]
SC_STD=[]

# list to store the statistical mearsure of RMSE 

RMS_MEAN=[]
RMS_MAX=[]
RMS_MIN=[]
RMS_IQR=[]
RMS_STD=[]

# List of all the coefficient 
maincoeff_disgust=[]

for i in range(200):

    audio_data,sr=waveplot(file_path_disgust[i])
    MFCCs_coeff=MFCCS_feature(audio_data,sr)
    maincoeff_disgust.append(MFCCs_coeff)
    
    #RMSE
    R=total_rms(audio_data)
    Audio_data_N =audio_data/R #for normalized each audio in old-fear category
    
    MEAN,MAX,MIN,IQR,SD=RMS_(Audio_data_N,1024,512)
    
    RMS_MEAN.append(MEAN)
    RMS_MAX.append(MAX)
    RMS_MIN.append(MIN)
    RMS_IQR.append(IQR)
    RMS_STD.append(SD)
    
    # Spectral Centroid
    S_MEAN,S_MAX,S_MIN,S_iqr,S_sd=Spectral_freq(audio_data,1024,512 )
    SC_MEAN.append(S_MEAN)
    SC_MAX.append(S_MAX)
    SC_MIN.append(S_MIN)
    SC_IQR.append(S_iqr)
    SC_STD.append(S_sd)
    
# Making the Dictionary of RMS Statistical values and Spectral Centroid statistical Values 

dic={"MEAN_RMSE":RMS_MEAN,"MAX_RMSE":RMS_MAX,"MIN_RMSE":RMS_MIN,"IQR_RMSE":RMS_IQR,"STD_RMSE":RMS_STD,"MEAN_SC":SC_MEAN,"MAX_SC":SC_MAX,"MIN_SC":SC_MIN,"IQR_SC":SC_IQR,"STD_SC":SC_STD}

Df_r_s=pd.DataFrame(data=dic)
    
       
#print(MFCCs_coeff)    
#maincoeff_disgust

# Making a Dataframe of the given Audio 

Df_mfcc=pd.DataFrame(data=maincoeff_disgust,columns=("C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13"))
Df_mfcc['Label']="DISGUST"
Df_mfcc

Df_disgust=pd.concat([Df_r_s, Df_mfcc], axis=1)

Df_disgust 

,MEAN_RMSE,MAX_RMSE,MIN_RMSE,IQR_RMSE,STD_RMSE,MEAN_SC,MAX_SC,MIN_SC,IQR_SC,STD_SC,...,C5,C6,C7,C8,C9,C10,C11,C12,C13,Label
0,0.766622,2.614507,0.081673,0.750718,0.636873,1979.622416,6760.998357,377.511273,1056.302865,1750.535218,...,11.983032,4.657784,-6.131589,0.464807,-14.900079,3.433928,-9.463342,2.520785,0.688317,DISGUST
1,0.738766,3.422107,0.021294,0.732282,0.673115,2088.608954,6386.426671,387.865620,729.911672,1667.558459,...,6.329151,2.994816,-6.211717,3.615574,-14.892809,4.831861,-7.826244,4.640342,-0.084397,DISGUST
2,0.746432,3.364934,0.036874,0.669256,0.662354,2342.724110,6633.260431,323.105892,1601.732969,2043.311828,...,17.024797,7.066236,-11.459501,0.679578,-13.489285,-4.292565,-9.749439,1.388700,0.096857,DISGUST
3,0.758673,3.266282,0.049000,0.880164,0.648797,2001.080349,6768.841245,362.802304,925.917759,1783.583115,...,14.110828,6.779713,-12.435480,4.000092,-11.875082,5.917932,-7.463096,2.873588,-2.536114,DISGUST
4,0.723753,2.913686,0.032139,0.612216,0.683282,1861.634379,6566.330537,297.908099,917.348158,1803.910729,...,15.283918,6.421708,-5.558183,5.823524,-15.407926,-9.349440,-11.381513,-5.593702,-2.099773,DISGUST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.669490,3.348121,0.029364,0.446788,0.739708,1983.394232,6353.823946,399.051561,722.295335,1847.799227,...,11.845413,0.630728,-15.611544,-2.868850,-16.218504,-1.987069,-11.593783,-0.787793,-1.799227,DISGUST
196,0.753772,3.049748,0.049382,0.629042,0.656133,1673.903844,6306.408659,399.870664,498.021850,1633.741443,...,10.752189,16.351158,-4.501810,-12.025866,-18.813480,-1.716978,-14.615490,-2.648144,-2.616892,DISGUST
197,0.717264,3.244007,0.051705,0.414405,0.691310,2392.941715,6548.706683,425.478173,1353.895747,1915.654490,...,12.017066,10.636068,-8.344769,1.798151,-17.749025,5.569857,-11.815313,6.267455,-4.471897,DISGUST
198,0.734317,3.069519,0.026629,0.562223,0.672190,1888.907021,6443.170733,423.116159,668.290578,1750.783853,...,11.103639,3.412206,-8.069880,-1.655515,-19.228628,-4.254327,-11.501009,-3.268635,-2.182376,DISGUST


In [ ]:
#MFCCs & its Derivatives of first Audio in old-pleasant surprise category ---
#07
# list to store the statistical mearsure of Spectral Centroid 

SC_MEAN=[]
SC_MAX=[]
SC_MIN=[]
SC_IQR=[]
SC_STD=[]

# list to store the statistical mearsure of RMSE 

RMS_MEAN=[]
RMS_MAX=[]
RMS_MIN=[]
RMS_IQR=[]
RMS_STD=[]

# List of all the coefficient 

maincoeff_PS=[]

for i in range(200):

    audio_data,sr=waveplot(file_path_PS[i])
    MFCCs_coeff=MFCCS_feature(audio_data,sr)
    maincoeff_PS.append(MFCCs_coeff)
    
    #RMSE
    R=total_rms(audio_data)
    Audio_data_N =audio_data/R #for normalized each audio in old-fear category
    
    MEAN,MAX,MIN,IQR,SD=RMS_(Audio_data_N,1024,512)
    
    RMS_MEAN.append(MEAN)
    RMS_MAX.append(MAX)
    RMS_MIN.append(MIN)
    RMS_IQR.append(IQR)
    RMS_STD.append(SD)
    
    # Spectral Centroid
    S_MEAN,S_MAX,S_MIN,S_iqr,S_sd=Spectral_freq(audio_data,1024,512 )
    SC_MEAN.append(S_MEAN)
    SC_MAX.append(S_MAX)
    SC_MIN.append(S_MIN)
    SC_IQR.append(S_iqr)
    SC_STD.append(S_sd)
    
# Making the Dictionary of RMS Statistical values and Spectral Centroid statistical Values 

dic={"MEAN_RMSE":RMS_MEAN,"MAX_RMSE":RMS_MAX,"MIN_RMSE":RMS_MIN,"IQR_RMSE":RMS_IQR,"STD_RMSE":RMS_STD,"MEAN_SC":SC_MEAN,"MAX_SC":SC_MAX,"MIN_SC":SC_MIN,"IQR_SC":SC_IQR,"STD_SC":SC_STD}

Df_r_s=pd.DataFrame(data=dic)
       
#print(MFCCs_coeff)    
#maincoeff_pleasant surprise

# Making a Dataframe of the given Audio 

Df_mfcc=pd.DataFrame(data=maincoeff_PS,columns=("C1","C2","C3","C4","C5","C6","C7","C8","C9","C10","C11","C12","C13"))
Df_mfcc['Label']="PLeasant Surprise"
Df_mfcc

Df_ps=pd.concat([Df_r_s, Df_mfcc], axis=1)

Df_ps

,MEAN_RMSE,MAX_RMSE,MIN_RMSE,IQR_RMSE,STD_RMSE,MEAN_SC,MAX_SC,MIN_SC,IQR_SC,STD_SC,...,C5,C6,C7,C8,C9,C10,C11,C12,C13,Label
0,0.836992,2.198855,0.057750,0.925569,0.542305,1964.282646,6743.298908,435.543689,1097.149018,1653.579432,...,8.578238,0.709812,-9.753245,2.768565,-14.067597,1.298902,-12.837076,-3.303814,-1.943674,PLeasant Surprise
1,0.776805,3.355134,0.022853,0.773351,0.623190,1767.906730,6245.007536,295.215807,901.610304,1547.876395,...,-0.771864,2.442353,-7.021891,1.362751,-15.447729,2.849486,-14.891939,-2.639144,0.848396,PLeasant Surprise
2,0.731650,3.446321,0.030727,0.777612,0.675480,2437.252344,6877.079999,320.205359,1418.563503,1965.585156,...,7.982354,-4.547149,-12.660566,2.594647,-14.942225,0.915480,-14.432895,-2.982659,0.312145,PLeasant Surprise
3,0.830158,2.194178,0.035574,0.856276,0.551186,2164.763559,6337.616286,400.146185,1039.817817,1730.115347,...,3.931713,6.273214,-9.535351,3.502368,-15.553627,6.401196,-11.678422,-4.353988,-2.250307,PLeasant Surprise
4,0.857692,2.026796,0.089469,0.718908,0.513717,1777.429054,5991.172848,354.677147,1042.588013,1416.271000,...,4.960605,1.458279,-12.546000,0.631655,-23.882570,-12.307263,-6.473175,-13.034305,-0.895717,PLeasant Surprise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.776934,2.680340,0.013900,0.847367,0.621394,2218.493047,6264.785479,468.630464,1325.269854,1673.758066,...,4.232765,-5.246085,-16.235121,-2.397815,-16.905508,-3.825086,-13.813653,-1.870977,-0.031041,PLeasant Surprise
196,0.836154,2.266734,0.034036,0.736053,0.540618,2117.415911,6528.922272,537.039054,716.646769,1614.980860,...,9.542933,3.974788,-13.788242,-7.236237,-17.329779,1.623405,-21.575628,-8.341942,-5.100511,PLeasant Surprise
197,0.785901,2.227862,0.010713,0.916694,0.611746,2510.453854,6473.823245,509.923165,1438.037046,1880.229078,...,1.898260,-5.140545,-22.510574,2.165768,-15.491080,-2.059357,-14.796926,-4.694530,-3.759768,PLeasant Surprise
198,0.851966,2.131936,0.022060,0.847358,0.519802,1907.394336,6139.862964,489.082150,712.165503,1663.745624,...,3.496547,-0.303660,-10.765730,2.465810,-17.037273,-7.782178,-15.882672,-6.776903,-8.799692,PLeasant Surprise


## Combining the features for all the Emotions

In [ ]:
Comb_DF=pd.concat([Df_fear,Df_happy,Df_neutral,Df_sad,Df_angry,Df_disgust,Df_ps],axis=0,ignore_index=True)
Comb_DF

,MEAN_RMSE,MAX_RMSE,MIN_RMSE,IQR_RMSE,STD_RMSE,MEAN_SC,MAX_SC,MIN_SC,IQR_SC,STD_SC,...,C5,C6,C7,C8,C9,C10,C11,C12,C13,Label
0,0.771957,2.092660,0.014741,1.228814,0.629168,1962.031387,6378.780076,402.558634,608.917698,1509.051985,...,2.283087,3.844167,-14.228564,0.269647,-11.322652,4.499730,-13.458061,-6.326655,-4.551631,FEAR
1,0.817040,2.150111,0.027914,0.838567,0.573831,1942.036062,6370.429580,510.573035,557.204637,1538.834203,...,-7.444870,5.448284,-12.487878,0.189825,-16.319094,1.524716,-21.328081,-3.633686,-6.368850,FEAR
2,0.785567,2.242551,0.017252,1.188739,0.613742,2442.161223,6223.610055,450.629524,1456.653181,1800.796830,...,7.653680,0.787712,-13.500839,-1.832727,-12.877916,1.519425,-18.313948,-6.033486,-3.560616,FEAR
3,0.781189,2.286831,0.012800,1.080863,0.618558,2357.061960,6657.249797,385.559075,1540.755298,1791.091198,...,-2.028609,3.328899,-12.075669,-0.014596,-15.432942,4.545129,-14.551147,-4.555892,-8.357164,FEAR
4,0.817185,1.942002,0.026914,0.911707,0.565479,1914.672326,5802.151158,460.611570,743.397902,1485.758096,...,8.218389,1.294082,-7.010327,2.554072,-14.801518,-2.347218,-18.474848,-5.801582,-2.754475,FEAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,0.776934,2.680340,0.013900,0.847367,0.621394,2218.493047,6264.785479,468.630464,1325.269854,1673.758066,...,4.232765,-5.246085,-16.235121,-2.397815,-16.905508,-3.825086,-13.813653,-1.870977,-0.031041,PLeasant Surprise
1396,0.836154,2.266734,0.034036,0.736053,0.540618,2117.415911,6528.922272,537.039054,716.646769,1614.980860,...,9.542933,3.974788,-13.788242,-7.236237,-17.329779,1.623405,-21.575628,-8.341942,-5.100511,PLeasant Surprise
1397,0.785901,2.227862,0.010713,0.916694,0.611746,2510.453854,6473.823245,509.923165,1438.037046,1880.229078,...,1.898260,-5.140545,-22.510574,2.165768,-15.491080,-2.059357,-14.796926,-4.694530,-3.759768,PLeasant Surprise
1398,0.851966,2.131936,0.022060,0.847358,0.519802,1907.394336,6139.862964,489.082150,712.165503,1663.745624,...,3.496547,-0.303660,-10.765730,2.465810,-17.037273,-7.782178,-15.882672,-6.776903,-8.799692,PLeasant Surprise


In [ ]:
# Changing a dataframe into CSV file
Comb_DF.to_csv("Combined_Audio_features.csv")
